In [13]:

import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image


In [14]:
# params
dataset_path = 'archive/images'
dataset_path2 = 'ar1'

img_height, img_width = 150, 150
batch_size = 32
epochs = 20

data = []
labels = []

In [15]:
#phone
for filename in os.listdir(dataset_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        data.append(os.path.join(dataset_path, filename))
        labels.append(1)

In [16]:
#not 
for filename in os.listdir(dataset_path2):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        data.append(os.path.join(dataset_path2, filename))
        labels.append(0)  

In [17]:
train_data, val_data, train_labels, val_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

In [18]:
print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")

Training samples: 400, Validation samples: 100


In [19]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Трансформации для предобработки изображений
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = CustomDataset(train_data, train_labels, transform=transform)
val_dataset = CustomDataset(val_data, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [20]:
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, layers):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()
        input_channels = 3  # Используем 3 канала (RGB)

        for output_channels in layers:
            self.layers.append(nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(2))
            input_channels = output_channels

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_channels * (img_height // 2**len(layers))**2, 128),
            nn.ReLU(),
            nn.Linear(128, 2)  # Используем 2 выхода для бинарной классификации
        )

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.classifier(x)
        return x

In [21]:
model = CustomCNN([64, 128])
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [22]:
def train_and_validate(model, train_loader, val_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs} training complete.')

    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Validation Accuracy: {val_accuracy:.2f}%')

    return model


In [23]:
# Обучение модели
final_model = train_and_validate(model, train_loader, val_loader, optimizer, criterion, epochs=epochs)


Epoch 1/20 training complete.
Epoch 2/20 training complete.
Epoch 3/20 training complete.
Epoch 4/20 training complete.
Epoch 5/20 training complete.
Epoch 6/20 training complete.
Epoch 7/20 training complete.
Epoch 8/20 training complete.
Epoch 9/20 training complete.
Epoch 10/20 training complete.
Epoch 11/20 training complete.
Epoch 12/20 training complete.
Epoch 13/20 training complete.
Epoch 14/20 training complete.
Epoch 15/20 training complete.
Epoch 16/20 training complete.
Epoch 17/20 training complete.
Epoch 18/20 training complete.
Epoch 19/20 training complete.
Epoch 20/20 training complete.
Validation Accuracy: 72.00%


In [37]:
image_path = 'photo_3_2024-06-19_14-39-11.jpg'

In [38]:
from PIL import Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Добавляем размерность пакета
    return image

In [39]:
def predict_image(model, image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return predicted.item()


In [40]:
prediction = predict_image(model, image_path)
predicted_class = 'Phone' if prediction == 1 else 'No phone'
print(f'Prediction: {predicted_class}')

Prediction: No phone


In [41]:
torch.save(final_model.state_dict(), 'final_best_model.pth')